In [1]:
%matplotlib inline

from multiprocessing import Pool

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import itertools
from itertools import permutations

# Top k-Ranking 

class MAB:
    def __init__(self, arms_len, max_k, mode):
        self.arms_len = arms_len
        self.max_k = max_k
        self.mode = mode
        
        self.rewards = []
        self.arms_reward = np.zeros(self.arms_len)
        self.arms_expectation = .00001*np.random.random_sample((self.arms_len,)) 
        self.arms_expectation_counts = np.zeros(self.arms_len)
        
    def choose_arms(self):
        if self.mode == 'random':
            return np.random.permutation(self.arms_len)[0:self.max_k]
        elif self.mode == 'epsilon_greedy1':
            if np.random.random_sample() < .1:
                return np.random.permutation(self.arms_len)[0:self.max_k]
            else:
                arms_expectation_sorted_index = np.flip(np.argsort(self.arms_expectation))
                return arms_expectation_sorted_index[0:self.max_k]
        elif self.mode == 'epsilon_greedy2':
            if np.random.random_sample() < .5:
                return np.random.permutation(self.arms_len)[0:self.max_k]
            else:
                arms_expectation_sorted_index = np.flip(np.argsort(self.arms_expectation))
                return arms_expectation_sorted_index[0:self.max_k]
        elif self.mode == 'epsilon_greedy3':
            if np.random.random_sample() < .9:
                return np.random.permutation(self.arms_len)[0:self.max_k]
            else:
                arms_expectation_sorted_index = np.flip(np.argsort(self.arms_expectation))
                return arms_expectation_sorted_index[0:self.max_k]
        elif self.mode == 'max_expectation':
            arms_expectation_sorted_index = np.flip(np.argsort(self.arms_expectation))
            return arms_expectation_sorted_index[0:self.max_k]
        elif self.mode == 'optimal':
            if self.max_k == 1:
                if 5 < self.arms_len:
                    return np.array([5])
                else:
                    return self.arms_len-1
            elif self.max_k == 2:
                if 6 < self.arms_len:
                    return np.array([5, 6])
                else:
                    return np.array(range(self.arms_len-2, self.arms_len))
            elif self.max_k == 3:   
                if 6 < self.arms_len:
                    return np.array([4, 5, 6])
                else:
                    return np.array(range(self.arms_len-3, self.arms_len))
            elif self.max_k == 4:
                if 7 < self.arms_len:
                    return np.array([4, 5, 6, 7])
                else:
                    return np.array(range(self.arms_len-4, self.arms_len))
            elif self.max_k == 5:
                if 7 < self.arms_len:
                    return np.array([3, 4, 5, 6, 7])
                else:
                    return np.array(range(self.arms_len-5, self.arms_len))
            elif self.max_k == 6:
                if 8 < self.arms_len:
                    return np.array([3, 4, 5, 6, 7, 8])
                else:
                    return np.array(range(self.arms_len-6, self.arms_len))
        elif 'ucb' in self.mode:
            arms_std = np.zeros(self.arms_len)
            for i in range(self.arms_len):
                for j in range(len(self.rewards)):
                    if i in self.rewards[j][0]:
                        arms_std[i] = arms_std[i] + np.power(self.rewards[j][1] - self.arms_expectation[i], 2)
            arms_std = np.power(arms_std,1/2)

            arms_ucb = np.zeros(self.arms_len)
            for i in range(self.arms_len):
                if self.arms_expectation_counts[i]>0:
                    if self.mode == 'ucb':
                        arms_ucb[i] = self.arms_expectation[i] + arms_std[i]/self.arms_expectation_counts[i]
                    if self.mode == 'ucb2':
                        arms_ucb[i] = self.arms_expectation[i] + arms_std[i]*np.maximum(0, 30-self.arms_expectation_counts[i])
                else:
                    arms_ucb[i] = self.arms_expectation[i] + arms_std[i]
                
            arms_ucb_sorted_index = np.flip(np.argsort(arms_ucb))
            return arms_ucb_sorted_index[0:self.max_k]
        else:
            print('choose_arms unknown mode error')
    
    def set_reward(self, arms, reward):
        self.rewards.append((arms, reward))
        for arm in arms:
            arm = int(arm)
            self.arms_reward[arm] += reward
            self.arms_expectation_counts[arm] += 1
            self.arms_expectation[arm] = self.arms_reward[arm]/self.arms_expectation_counts[arm]

def test_model_reward(MAB,reward_noise,epochs):

    reward_vector = np.zeros(epochs)

    for i in range(epochs):

        arms = MAB.choose_arms()
        error = np.zeros(arms.shape[0])
        for j in range(arms.shape[0]):
            arm = arms[j]
            error[j] = np.absolute(arm - 5)
        total_error = np.sum(error)
        # if MAB.max_k == 1:
        #     total_error = np.sum(error)
        # elif MAB.max_k == 2:
        #     total_error = np.sum(error)-1
        # elif MAB.max_k == 3:
        #     total_error = np.sum(error)-2
        # elif MAB.max_k == 4:
        #     total_error = np.sum(error)-4
        # elif MAB.max_k == 5:
        #     total_error = np.sum(error)-6
        # elif MAB.max_k == 6:
        #     total_error = np.sum(error)-9
        total_error = total_error + np.abs(np.random.randn()*reward_noise)
        reward = -total_error
        MAB.set_reward(arms, reward)
        if i>0:
            reward_vector[i] = reward + reward_vector[i-1]
        else:
            reward_vector[i] = reward

    for i in range(epochs):
        reward_vector[i] = reward_vector[i]/(i+1)

    return reward_vector

def test_model_reward_wrapper(args):
    arms = args[0]
    k = args[1]
    method = args[2]
    reward_noise = args[3]
    epochs = args[4]
    MAB_method = MAB(arms,k,method)
    return test_model_reward(MAB_method,reward_noise,epochs)

    # q.put(1)

    # conn.send(test_model_reward(MAB_method,reward_noise,epochs))
    # conn.send(1)
    # conn.close()

def test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample):
    for arms in arms_range:
        for reward_noise in reward_noise_range:
            for k in k_range:
                for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
                    reward_method = np.zeros((stat_sample,epochs))

                    parallel_args = []
                    for j in range(stat_sample):
                        parallel_args.append([arms,k,method,reward_noise,epochs])

                    reward_method_wrapper = list(map(test_model_reward_wrapper, iter(parallel_args)))

                    with Pool(5) as p:
                        print(p.map(test_model_reward_wrapper, iter(parallel_args)))

                    # map(test_model_reward_wrapper, (arms,k,method,)))

                    # print(1)


                    # q = SimpleQueue()
                    
                    # parallel_processes = []
                    # for j in range(stat_sample):
                        
                    #     parent_conn, child_conn = Pipe()
                    #     p = Process(target=test_model_reward_wrapper, args=(child_conn,arms,k,method,))
                    #     p.start()
                    #     parallel_processes.append([parent_conn,p])

                    # with Pool(1) as p:
                    #     reward_method_wrapper = p.map(test_model_reward_wrapper, parallel_args)
                    # print(reward_method_wrapper)

                    # print(3)

                    # reward_method_wrapper = []
                    # for j in range(stat_sample):
                    #     parent_conn = parallel_processes[j][0]
                    #     reward_method_wrapper.append(parent_conn.recv())

                    # print(4)
                    
                    # for j in range(stat_sample):
                    #     p = parallel_processes[j][1]
                    #     p.join()

                    # print(2)
                    
                    for j in range(stat_sample):
                        reward_method[j,:] = reward_method_wrapper[j]
                    
                    # for j in range(stat_sample):
                    #     MAB_method = MAB(arms,k,None,method)

                    #     reward_method[j,:] = test_model_reward(MAB_method,reward_noise,epochs)

                    reward_method_avg = np.mean(reward_method, axis=0)
                    reward_method_std = np.std(reward_method, axis=0)

                    plt.errorbar(range(epochs),reward_method_avg,yerr=reward_method_std/5.)
                
                plt.title('Average Cumulative Reward of Top '+str(k)+'-Ranking of '+str(arms)+' arms, noise:'+str(reward_noise))
                plt.legend(['max_expectation'.replace('_',' '),
                            'epsilon_greedy 0.1'.replace('_',' '),
                            'epsilon_greedy 0.5'.replace('_',' '),
                            'epsilon_greedy 0.9'.replace('_',' '),
                            'random','ucb','ucb2','optimal'])
                plt.savefig('reward,k='+str(k)+',arms='+str(arms)+',noise='+str(reward_noise)+'.png')
                plt.show()


In [2]:
arms_range = [10]
reward_noise_range = [1/2]
epochs = 100
k_range_max = 6
k_range = range(1,k_range_max+1)
stat_sample = 50
test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample)


In [ ]:
arms_range = [6]
reward_noise_range = [1/2]
epochs = 100
k_range_max = 6
k_range = range(1,k_range_max+1)
stat_sample = 50
test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample)


In [ ]:
# arms = 6
# reward_noise = 1/2
# epochs = 100
# k_range = 6
# stat_sample = 50
# for k in range(1,k_range+1):
#     for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
#         reward_index = np.zeros((stat_sample,epochs))
#         reward_method = np.zeros((stat_sample,epochs))
#         reward_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             reward_index[j,:] = range(epochs)
#             reward_method[j,:] = test_model_reward(MAB_method,reward_noise,epochs)
#             reward_optimal[j,:] = test_model_reward(MAB_optimal,reward_noise,epochs)

#         reward_method_avg = np.mean(reward_method, axis=0)
#         reward_method_std = np.std(reward_method, axis=0)

#         plt.errorbar(range(epochs),reward_method_avg,yerr=reward_method_std/5.)
    
#     plt.title('Average Cumulative Reward of Top '+str(k)+'-Ranking of '+str(arms)+' arms, noise:'+str(reward_noise))
#     plt.legend(['max_expectation'.replace('_',' '),
#                 'epsilon_greedy 0.1'.replace('_',' '),
#                 'epsilon_greedy 0.5'.replace('_',' '),
#                 'epsilon_greedy 0.9'.replace('_',' '),
#                 'random','ucb','ucb2','optimal'])
#     plt.show()

In [ ]:
arms_range = [20]
reward_noise_range = [1/2]
epochs = 100
k_range_max = 6
k_range = range(1,k_range_max+1)
stat_sample = 50
test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample)


In [ ]:
# arms = 20
# reward_noise = 1/2
# epochs = 100
# k_range = 6
# stat_sample = 50
# for k in range(1,k_range+1):
#     for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
#         reward_index = np.zeros((stat_sample,epochs))
#         reward_method = np.zeros((stat_sample,epochs))
#         reward_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             reward_index[j,:] = range(epochs)
#             reward_method[j,:] = test_model_reward(MAB_method,reward_noise,epochs)
#             reward_optimal[j,:] = test_model_reward(MAB_optimal,reward_noise,epochs)

#         reward_method_avg = np.mean(reward_method, axis=0)
#         reward_method_std = np.std(reward_method, axis=0)

#         plt.errorbar(range(epochs),reward_method_avg,yerr=reward_method_std/5.)
    
#     plt.title('Average Cumulative Reward of Top '+str(k)+'-Ranking of '+str(arms)+' arms, noise:'+str(reward_noise))
#     plt.legend(['max_expectation'.replace('_',' '),
#                 'epsilon_greedy 0.1'.replace('_',' '),
#                 'epsilon_greedy 0.5'.replace('_',' '),
#                 'epsilon_greedy 0.9'.replace('_',' '),
#                 'random','ucb','ucb2','optimal'])
#     plt.show()

In [ ]:
arms_range = [6, 10,15,25,50,100]
reward_noise_range = [1/2]
epochs = 100
k_range = [3]
stat_sample = 50
test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample)



In [ ]:
# reward_noise = 1/2
# epochs = 100
# k = 3
# stat_sample = 50
# for arms in [6, 10,25,50,100]:
#     for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
#         reward_index = np.zeros((stat_sample,epochs))
#         reward_method = np.zeros((stat_sample,epochs))
#         reward_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             reward_index[j,:] = range(epochs)
#             reward_method[j,:] = test_model_reward(MAB_method,reward_noise,epochs)
#             reward_optimal[j,:] = test_model_reward(MAB_optimal,reward_noise,epochs)

#         reward_method_avg = np.mean(reward_method, axis=0)
#         reward_method_std = np.std(reward_method, axis=0)

#         plt.errorbar(range(epochs),reward_method_avg,yerr=reward_method_std/5.)
    
#     plt.title('Average Cumulative Reward of Top '+str(k)+'-Ranking of '+str(arms)+' arms, noise:'+str(reward_noise))
#     plt.legend(['max_expectation'.replace('_',' '),
#                 'epsilon_greedy 0.1'.replace('_',' '),
#                 'epsilon_greedy 0.5'.replace('_',' '),
#                 'epsilon_greedy 0.9'.replace('_',' '),
#                 'random','ucb','ucb2','optimal'])
#     plt.show()

In [ ]:
arms_range = [10]
reward_noise_range = [.01, .1, .5, 1, 10, 100]
epochs = 100
k_range = [3]
stat_sample = 50
test_plot_reward(arms_range,reward_noise_range,epochs,k_range,stat_sample)


In [ ]:
# arms = 10
# reward_noise = 1/2
# epochs = 100
# k = 3
# stat_sample = 50
# for reward_noise in [.01, .1, .5, 1, 10, 100]:
#     for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
#         reward_index = np.zeros((stat_sample,epochs))
#         reward_method = np.zeros((stat_sample,epochs))
#         reward_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             reward_index[j,:] = range(epochs)
#             reward_method[j,:] = test_model_reward(MAB_method,reward_noise,epochs)
#             reward_optimal[j,:] = test_model_reward(MAB_optimal,reward_noise,epochs)

#         reward_method_avg = np.mean(reward_method, axis=0)
#         reward_method_std = np.std(reward_method, axis=0)

#         plt.errorbar(range(epochs),reward_method_avg,yerr=reward_method_std/5.)
    
#     plt.title('Average Cumulative Reward of Top '+str(k)+'-Ranking of '+str(arms)+' arms, noise:'+str(reward_noise))
#     plt.legend(['max_expectation'.replace('_',' '),
#                 'epsilon_greedy 0.1'.replace('_',' '),
#                 'epsilon_greedy 0.5'.replace('_',' '),
#                 'epsilon_greedy 0.9'.replace('_',' '),
#                 'random','ucb','ucb2','optimal'])
#     plt.show()

In [ ]:
# arms = 10
# reward_noise = 1/1
# epochs = 50
# k_range = 5
# stat_sample = 5
# for method in ['max_expectation','epsilon_greedy','random','ucb','ucb2']:
#     for k in range(1,k_range):
#         regret_index = np.zeros((stat_sample,epochs))
#         regret_method = np.zeros((stat_sample,epochs))
#         regret_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             regret_index[j,:] = range(epochs)
#             regret_method[j,:] = test_model(MAB_method,reward_noise,epochs)
#             regret_optimal[j,:] = test_model(MAB_optimal,reward_noise,epochs)

#         plt.plot(np.transpose(regret_index),np.transpose(regret_method),
#                     np.transpose(regret_index),np.transpose(regret_optimal))

#         plt.title('Regret of Top '+str(k)+'-Ranking: '+method.replace('_',' '))
#         plt.show()

In [ ]:
# arms = 10
# reward_noise = 1/2
# epochs = 100
# k_range = 5
# stat_sample = 50
# for k in range(1,k_range):
#     for method in ['max_expectation','epsilon_greedy1','epsilon_greedy2','epsilon_greedy3','random','ucb','ucb2','optimal']:
#         regret_index = np.zeros((stat_sample,epochs))
#         regret_method = np.zeros((stat_sample,epochs))
#         regret_optimal = np.zeros((stat_sample,epochs))
#         for j in range(stat_sample):
#             MAB_method = MAB(arms,k,None,method)
#             MAB_optimal = MAB(arms,k,None,'optimal')

#             regret_index[j,:] = range(epochs)
#             regret_method[j,:] = test_model(MAB_method,reward_noise,epochs)
#             regret_optimal[j,:] = test_model(MAB_optimal,reward_noise,epochs)

#         regret_method_avg = np.mean(regret_method, axis=0)
#         regret_method_std = np.std(regret_method, axis=0)

#         plt.errorbar(range(epochs),regret_method_avg,yerr=regret_method_std/5.)
    
#     plt.title('Regret of Top '+str(k)+'-Ranking')
#     plt.legend(['max_expectation'.replace('_',' '),
#                 'epsilon_greedy 0.1'.replace('_',' '),
#                 'epsilon_greedy 0.5'.replace('_',' '),
#                 'epsilon_greedy 0.9'.replace('_',' '),
#                 'random','ucb','ucb2','optimal'])
#     plt.show()

In [ ]:
# arms = 10
# reward_noise = 1/1
# epochs = 50
# k_range = 5
# for k in range(1,k_range):
#     MAB_max_expectation = MAB(arms,k,None,'max_expectation')
#     MAB_epsilon_greedy = MAB(arms,k,None,'epsilon_greedy')
#     MAB_random = MAB(arms,k,None,'random')
#     MAB_optimal = MAB(arms,k,None,'optimal')
#     MAB_ucb = MAB(arms,k,None,'ucb')
#     MAB_ucb2 = MAB(arms,k,None,'ucb2')

#     regret_max_expectation = test_model(MAB_max_expectation,reward_noise,epochs)
#     regret_epsilon_greedy = test_model(MAB_epsilon_greedy,reward_noise,epochs)
#     regret_random = test_model(MAB_random,reward_noise,epochs)
#     regret_optimal = test_model(MAB_optimal,reward_noise,epochs)
#     regret_ucb = test_model(MAB_ucb,reward_noise,epochs)
#     regret_ucb2 = test_model(MAB_ucb2,reward_noise,epochs)

#     plt.plot(range(len(regret_max_expectation)),regret_max_expectation,
#              range(len(regret_epsilon_greedy)),regret_epsilon_greedy,
#              range(len(regret_random)),regret_random,
#              range(len(regret_optimal)),regret_optimal,
#              range(len(regret_ucb)),regret_ucb,
#              range(len(regret_ucb2)),regret_ucb2)
#     plt.legend(['max expectation','epsilon greedy','random','optimal','ucb','ucb2'])
#     plt.title('Regret of Top '+str(k)+'-Ranking')
#     plt.show()